In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys
import tensorflow_addons as tfa


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

# model = SwinTransformer3D(**cfg)

In [3]:
def get_model(num_classes,cfg, shape_of_input=(10,3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)
    x = backbone(inputs, training= False)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [4]:
shape_of_input = (5, 3, 8, 112,112)
model = get_model(5, cfg, shape_of_input=shape_of_input)

X = tf.random.normal(shape_of_input)
y = tf.random.uniform((shape_of_input[0],1), 0, 5, tf.dtypes.int32)
model.summary()

2022-09-12 04:34:29.851758: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-12 04:34:29.851848: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gsoc28gb): /proc/driver/nvidia/version does not exist
2022-09-12 04:34:29.853573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


shape_of_input:  (5, 3, 8, 112, 112)
5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 8, 112, 112)]  0         
                                                                 
 swin_transformer3d (SwinTra  (None, 768, 4, 4, 4)     29694438  
 nsformer3D)                                                     
                                                                 
 i3d_head_tf (I3DHead_tf)    (None, 5)                 3845      
                                                                 
Total params: 29,698,283
Trainable params: 27,854,315
Non-trainable params: 1,843,968
_________________________________________________________________


In [5]:
# inputs = tf.keras.Input(shape_of_input[1:])
# model(inputs, training=True)

In [6]:
y1 = model.predict(X)
y1

1/1 [==============================] - 6s 6s/step


array([[0.26121393, 0.2134705 , 0.1431237 , 0.19967034, 0.18252166],
       [0.25058764, 0.19801636, 0.15458485, 0.20697655, 0.18983455],
       [0.2401445 , 0.21149504, 0.16213812, 0.20081998, 0.18540232],
       [0.26834154, 0.20051232, 0.14415093, 0.21378434, 0.17321084],
       [0.2764194 , 0.21138228, 0.14738142, 0.18487862, 0.17993824]],
      dtype=float32)

In [7]:
optimizer = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=3e-4, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)

## pariwise constant decay

In [8]:


metrics=["top_k_categorical_accuracy", "categorical_accuracy"] 
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=optimizer, metrics=metrics)

model.fit(X, y, epochs=2)

Epoch 1/2
1/1 [==============================] - 23s 23s/step - loss: 1.6867 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8000
Epoch 2/2
1/1 [==============================] - 2s 2s/step - loss: 0.9781 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.0000e+00


In [9]:
y1 = model.predict(X)
y1

1/1 [==============================] - 5s 5s/step


array([[0.04362928, 0.16783701, 0.08070742, 0.34368616, 0.36414015],
       [0.04380906, 0.15608679, 0.08128527, 0.353145  , 0.36567393],
       [0.04338022, 0.12835914, 0.08491397, 0.3416093 , 0.40173745],
       [0.04404173, 0.16992965, 0.07919259, 0.35085276, 0.3559833 ],
       [0.04552931, 0.12983981, 0.08353335, 0.40218642, 0.33891118]],
      dtype=float32)

In [ ]:
y2 = model(X, training=False)
y2

In [ ]:
model.layers[1].layers[5].weights